In [139]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from random import randint

os.chdir("/home/william/Desktop")

In [140]:
datosTicTacToe = pd.read_csv("tic-tac-toe.csv")

In [141]:
for x in datosTicTacToe:
    if x != " Gana-x":
        datosTicTacToe[x] = datosTicTacToe[x].astype('category')
datosTicTacToe.dtypes

cuadro-superior-izquierdo     category
 cuadro-superior-medio        category
 cuadro-superior-derecho      category
 cuadro-medio-izquierdo       category
 cuadro-medio-medio           category
  cuadro-medio-derecho        category
 cuadro-inferior-izquierdo    category
 cuadro-inferior-medio        category
 cuadro-inferior-derecho      category
 Gana-x                         object
dtype: object

In [142]:
datosTicTacToePosiblesResultados = list(set(datosTicTacToe.iloc[:, -1]))

In [143]:
class PrediccionBase:
    def __init__(self, datos):
        self.__datos = self.cleaning(datos)
        self.__precisionGlobal = 0
        self.__error_global = 0
        self.__verdaderosNegativos = 0
        self.__falsosPositivos = 0
        self.__falsosNegativos = 0
        self.__verdaderosPositivos = 0
        self.__reporte = 0
        self.__precision_category = 0

    @property
    def datos(self):
        return self.__datos

    @property
    def reporte(self):
        return self.__reporte

    def cleaning(self, datos):
        datos = datos.replace({'x': 0, 'o': 1, 'b': 2})
        return datos

    def entrenamiento(self):
        pass

    def generacionReporte(self, nombreDelModelo):
        dict = {
            "Modelo": [nombreDelModelo],
            "Precision Global": [self.__precisionGlobal],
            "Error Global": [self.__error_global],
            "Verdaderos Positivos": [self.__verdaderosPositivos],
            "Verdaderos Negativos": [self.__verdaderosNegativos],
            "Falsos Negativos": [self.__falsosNegativos],
            "Falsos Positivos": [self.__falsosPositivos]}
        self.__reporte = pd.DataFrame(dict).join(self.__precision_category)

    def analsis(self, MC, modelo):
        self.__verdaderosNegativos, self.__falsosPositivos, self.__falsosNegativos, self.__verdaderosPositivos = MC.ravel()
        self.__precisionGlobal = np.sum(MC.diagonal()) / np.sum(MC)
        self.__error_global = 1 - self.__precisionGlobal
        self.__precision_category = pd.DataFrame(MC.diagonal() / np.sum(MC, axis=1)).T
        self.__precision_category.columns = ["Precision Positiva (PP)", "Precision Negativa (PN)"]
        self.generacionReporte(modelo)
        return {"Matriz de Confusión": MC,
                "Precisión Global": self.__precisionGlobal,
                "Error Global": self.__error_global,
                "Precisión por categoría": self.__precision_category}

class PrediccionSVN(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, kernel = "linear", entrenamiento="Suport Vector Machine",train_size=0.8):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = SVC(kernel=kernel ,random_state=0)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        for k in indices:
             print("\n%s:\n%s" % (k, str(indices[k])))

class PrediccionRandomForest(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, n_estimators=10, entrenamiento="Bosques Aleatorios"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = RandomForestClassifier(n_estimators=n_estimators, random_state=0)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))

Como podemos ver en el modelo predictivo de abajo, el modelo SVN RBF.
Tenemos una precision global del 0.875 lo cual es muy bueno, ademas, tiene un error global muy bajo.

La precision positiva es de un 67%, y una prediccion del 100% para la prediccion negativa.

In [144]:
#posibles Kernel : ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
prediccionSVNTicTacRbf = PrediccionSVN(datos=datosTicTacToe)
prediccionSVNTicTacRbf.entrenamiento(kernel='rbf', entrenamiento="SVN RBF")



Matriz de Confusión:
[[ 50  24]
 [  0 118]]

Precisión Global:
0.875

Error Global:
0.125

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.675676                      1.0


En este otro caso podemos ver que el nucleo de sigmoide predice muy terrible, se puede ver en la precision global, y en que no esta prediciendo bien los verdaderos negativos

Tiene una precision alta, y un error global alto tambie, ademas las precisiones son muy malas tambien

In [145]:
prediccionSVNSigmoid = PrediccionSVN(datos=datosTicTacToe)
prediccionSVNSigmoid.entrenamiento(kernel='sigmoid',entrenamiento="SVN SIGMOID")


Matriz de Confusión:
[[ 2 72]
 [44 74]]

Precisión Global:
0.3958333333333333

Error Global:
0.6041666666666667

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.027027                 0.627119


Como podemos ver en el dataset de abajo, rectifica lo que dijimos arriba, el modelo de RBF, de acuerdo a los datos predice mucho mejor que la otra.

In [146]:
pd.concat([prediccionSVNTicTacRbf.reporte, prediccionSVNSigmoid.reporte])

,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN RBF,0.875000,0.125000,118,50,0,24,0.675676,1.000000
0,SVN SIGMOID,0.395833,0.604167,74,2,44,72,0.027027,0.627119


In [147]:
datosWine = pd.read_csv("wine.csv")

El modelo linear predice excelente tiene una precision global del 98% y un error global del 0.10%, ademas si vemos las precisiones positivas podemos ver que tiene un 99% y la negativa un 96%
Este modelo es una maravilla para la prediccion!

In [148]:
prediccionSVNWineLinear = PrediccionSVN(datos=datosWine)
prediccionSVNWineLinear.entrenamiento(kernel='linear', entrenamiento="SVN Linear", train_size=0.75)
prediccionSVNWineLinear.reporte


Matriz de Confusión:
[[1224    3]
 [  14  384]]

Precisión Global:
0.9895384615384616

Error Global:
0.010461538461538411

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.997555                 0.964824


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN Linear,0.989538,0.010462,384,1224,14,3,0.997555,0.964824


Por otra parte el modelo SIGMOIDE esta fallando un poco mas que el modelo anterior, como podemos ver en la precision global, y ademas como podemos ver la matriz de correlacion esta hablando muy mal de la forma en la que predice este modelo

In [149]:
prediccionSVNWineSigmoid = PrediccionSVN(datos=datosWine)
prediccionSVNWineSigmoid.entrenamiento(kernel='sigmoid', entrenamiento="SVN SIGMOID", train_size=0.75)
prediccionSVNWineSigmoid.reporte


Matriz de Confusión:
[[1034  193]
 [ 214  184]]

Precisión Global:
0.7495384615384615

Error Global:
0.2504615384615385

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.842706                 0.462312


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN SIGMOID,0.749538,0.250462,184,1034,214,193,0.842706,0.462312


Reafimando lo anterior podemos ver que el modelo linear es mucho mucho mejor que el modelo sigmoide en este caso.

In [150]:
pd.concat([prediccionSVNWineLinear.reporte, prediccionSVNWineSigmoid.reporte])


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN Linear,0.989538,0.010462,384,1224,14,3,0.997555,0.964824
0,SVN SIGMOID,0.749538,0.250462,184,1034,214,193,0.842706,0.462312


Hablando sobre las TAREAS anteriores por ejemplo como podemos ver aqui comparado con el Random Forest tambien es un poquito peor mas que todo en la precision global, y en las precisiones especificas.

In [151]:
prediccionRandomForest = PrediccionRandomForest(datosWine)
prediccionRandomForest.entrenamiento(train_size=0.75)
prediccionRandomForest.reporte


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,Bosques Aleatorios,0.995692,0.004308,392,1226,6,1,0.999185,0.984925


En comparacion con los data frame  de arriba, en el caso donde solo usamos las variables mas importantes, en especifico con el linear, empeora un poquito la precision global, las precisiones especificas

In [155]:
datosWine = pd.DataFrame(datosWine, columns=["volatil.acidez", "cloruros", "total.sulfuro.dioxido",
                                             "libre.sulfuro.dioxido", "sulfitos", "densidad", "tipo"])
prediccionSVNVariablesImportantes = PrediccionSVN(datosWine)
prediccionSVNVariablesImportantes.entrenamiento(kernel="linear",train_size=0.75)
prediccionSVNVariablesImportantes.reporte



Matriz de Confusión:
[[1211   16]
 [  23  375]]

Precisión Global:
0.976

Error Global:
0.02400000000000002

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                  0.98696                 0.942211


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,Suport Vector Machine,0.976,0.024,375,1211,23,16,0.98696,0.942211
